In [28]:
import numpy as np
import cv2
import os
import random

In [29]:
# выбрать <amount> случайных пар и применить к ним аугментацию
def get_random_images(fold_images, fold_masks, amount=1, size=(800, 800)):
    for i in range(amount):
        name = random.choice(os.listdir(fold_images))
        path_im = os.path.join(fold_images, name)
        path_mask = os.path.join(fold_masks, name)
        im = cv2.imread(path_im)
        mask = cv2.imread(path_mask)
        yield random.choice([rotate, reflect, crop, blur])(im, mask)

In [30]:
# доступные функции для аугментации
def rotate(im, mask):
    angle = random.randint(-180, 180)
    height, width, _  = im.shape
    rot_mat = cv2.getRotationMatrix2D((height / 2, width / 2), angle, 1.0)
    im = cv2.warpAffine(im, rot_mat, (height, width), flags=cv2.INTER_LINEAR)
    mask = cv2.warpAffine(mask, rot_mat, (height, width), flags=cv2.INTER_LINEAR)
    return im, mask

def reflect(im, mask):
    if bool(random.randint(0,1)):
        return im[::-1], mask[::-1]
    else:
        return cv2.flip(im, 1), cv2.flip(mask, 1)

def crop(im, mask):
    height, width, _ = im.shape
    x = np.random.randint(0, height / 2)
    w = np.random.randint(25, height / 2)
    y = np.random.randint(0, width / 2)
    h = np.random.randint(25, width / 2)
    im = im[x:x+w, y:y+h]
    mask = mask[x:x+w, y:y+h]
    return im, mask

def blur(im, mask):
    return cv2.medianBlur(im, 5), cv2.medianBlur(mask, 5)

def resize_(im, mask, size):
    return cv2.resize(im, size), cv2.resize(mask, size)

In [31]:
np.random.seed(435)
fold_images = "data/nails_segmentation/images"
fold_masks  = "data/nails_segmentation/labels"

gen = get_random_images(fold_images, fold_masks, amount=15)
nails = []
for i in range(15):
    nails.append(next(gen))

# показ из 6 набора заданий
ind_im = 0
while True:
    conc = np.concatenate(nails[ind_im], axis=1)
    cv2.imshow("Nails. Use arrows to change image. Esc to exit", conc)
    key = cv2.waitKeyEx(0)
    if key == 2424832 or key == ord(' ') or key == ord('a') or key == ord('A'):
        ind_im = (ind_im - 1) % len(nails)
    if key == 2555904 or key == ord('d') or key == ord('D'):
        ind_im = (ind_im + 1) % len(nails)
    if key == 27:
        break
cv2.destroyAllWindows()